# Evaluate Cycling Behavior

This notebook shows you how to calculate the temperature curves and linear regressions on the daily metrics of each heat pump. Afterwards the outlier detection is performed to identify the heat pumps which cycle atypically. Additionally, you learn how to calculate the utilization and energy intensity normalized by degree day to find the heat pumps with high and low energy consumption. This, however, requires that the heat pump power and heated floor area are known. For the purpose of this notebook, the meta data and extracted cycling metrics provided in the folder ```data/03_extracted_daily_metrics``` are used.

### Imports

In [1]:
# general imports 
import pandas as pd 
import numpy as np 
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# add dicrectories of this repo to system path for and then import necessary functions
sys.path.append(os.path.dirname(os.getcwd())+'/src') 

from utils import *
from helper import *


### Load Daily Metrics Data and Meta Data

In [2]:
# -------------------------------------------------------------------------
# Column definitions
# -------------------------------------------------------------------------

# define columns for timestamp and consumption
key_timestamp = 'Timestamp' # column identifier of the timestamp in the data frame containing the smart meter data 
key_consumption = 'Energy_kWh' # column identifier of the energy consumption in the data frame containing the smart meter data 
key_temperature = 'daily_avgTmp'  # column identifier of the daily average temperature in the data frame containing the temperature data

# -------------------------------------------------------------------------
# Load data
# -------------------------------------------------------------------------

# load meta data 
filepath = os.path.dirname(os.getcwd()) + '/data/03_extracted_daily_metrics/meta_data.csv'
df_meta = pd.read_csv(filepath)

# data of daily cycling metrics
filepath = os.path.dirname(os.getcwd()) + '/data/03_extracted_daily_metrics/daily_metrics.csv'
df_metrics = pd.read_csv(filepath)

# define column for the average daily temperature
key_temperature = 'daily_avgTmp'

display(df_meta)
display(df_metrics)

,ID,HP_Power_kW,Heated_Floor_Area,Building_Year,Cluster_Building_Age,Cluster_HP_Size,Cluster_Building_Area,Cluster_HP_Type
0,PCLAoIG5RQIvrhKO1ZeY,12.0,207.00,1972.0,medium,big,medium,ground source
1,WmMADGWWJ0wpryG30Mrg,2.9,262.20,2011.0,young,small,medium,air source
2,fv0SKHJXQUgF_dZPZawx,NaN,368.00,2003.0,young,unknown,big,ground source
3,jItPLTBhBFIOntFQk2eD,NaN,172.50,1992.0,young,unknown,small,ground source
4,1vSbB9CGbKgFnhDZ3yRg,4.0,140.30,2007.0,young,small,small,air source
...,...,...,...,...,...,...,...,...
498,x8Bqqwz_WIA4u1Dvi26n_,2.3,258.75,2005.0,young,small,medium,ground source
499,3lOhJprEMqoXS2CklTlUM,2.5,230.00,1951.0,medium,small,medium,air source
500,mliXe2KNaNkmKZMrrjX7f,3.7,287.50,1998.0,young,small,big,air source
501,Bu4uJ_ayRTUAR_Nx__eoZ,NaN,276.00,1990.0,young,unknown,big,air source


,ID,date,operating_hours,cycles,average_cycle_length_hours,Energy_kWh,ratio_cycles_operating_hours,daily_avgTmp,daily_maxTmp,daily_maxtemp,degree_day
0,09WRxJL9k7ol,2021-12-17,1.00,1,1.00,1.03,1.00,NaN,NaN,NaN,NaN
1,09WRxJL9k7ol,2021-12-18,24.00,1,24.00,25.68,0.04,NaN,NaN,NaN,NaN
2,09WRxJL9k7ol,2021-12-19,24.00,1,24.00,27.03,0.04,NaN,NaN,NaN,NaN
3,09WRxJL9k7ol,2021-12-20,21.03,3,7.01,26.74,0.14,NaN,NaN,NaN,NaN
4,09WRxJL9k7ol,2021-12-21,21.36,3,7.12,28.13,0.14,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
198794,zvNJQfjyMrMydMVP2gld,2022-09-17,8.98,13,0.69,37.75,1.45,10.0,7.0,12.0,10.0
198795,zvNJQfjyMrMydMVP2gld,2022-09-18,6.33,11,0.58,26.19,1.74,9.0,2.0,13.0,11.0
198796,zvNJQfjyMrMydMVP2gld,2022-09-19,5.52,10,0.55,21.73,1.81,11.0,2.0,15.0,9.0
198797,zvNJQfjyMrMydMVP2gld,2022-09-20,5.56,8,0.70,24.32,1.44,10.0,3.0,14.0,10.0


### Calculate Linear Regression on Temperature Curves
Perform this step for each heat pump and metric as the slopes and intercepts are needed in the next step as part of the outlier detection. 

In [3]:
# define settings for linear regression
metrics = ['operating_hours' , 'cycles', 'ratio_cycles_operating_hours', 'average_cycle_length_hours'] # the metrics for which to calculate linear regressions
temp_min = 0 # the minimum temperature to be considered for the temperature curves 
temp_max = 12 # the maximum temperature to be considered for the temperature curves
min_temps = 6 #  minimum number of different temperature values to be observed for linear regression - otherwise regression should not be calculated because it would be erroneous

# create the Helper object for extracting cycles 
helper = CycleHelper(key_timestamp, key_consumption)

# create a data frame that stores all parameters extracted 
df_fits = pd.DataFrame()

# loop over obserations of metrics
for id in tqdm(df_metrics['ID'].unique()): 

    # select the right observations 
    df_sub = df_metrics[df_metrics['ID'] == id]

    # ----------------------------------------------------
    # CALCULATE TEMPERATURE CURVE AND FIT LINEAR REGRESSION
    # ----------------------------------------------------

    # This part performs the following steps for each metric: 
    # - filter and only consider values in the given temperature range 
    # - ensure that the minimum number of temperatures at which observations are available is met
    # - calculate temperature curves (i.e. median values per outdoor temperature) 
    # - fit linear regression to the temperature curves 
    # - extract R^2 values, intercept and slope of the fitted linear model

    # calculate linear regression parameters for each metric
    df_params = helper.calculate_linear_regression_parameters_from_daily_metrics(df_sub, temp_col=key_temperature, metrics=metrics, temp_min=temp_min, temp_max=temp_max, min_temps=min_temps)

    # NOTE: the method above returns None if the parameters cannot be calculated (e.g., because of not enough observations in the given temperature range)
    if df_params is not None: 
        
        # add the ID to the data frame
        df_params.insert(0, 'ID', id)

        # add to all observations
        df_fits = pd.concat([df_fits, df_params])

    # reset index
    df_fits.reset_index(drop=True, inplace=True)

# ----------------------------------------------------
# SHOW RESULST
# ----------------------------------------------------

display(df_fits)

100%|██████████| 503/503 [00:14<00:00, 34.55it/s]


,ID,metric,R2,intercept,slope
0,0K0Xq95kh0c51N0x2pB1,operating_hours,0.984440,13.485275,-0.624341
1,0K0Xq95kh0c51N0x2pB1,cycles,0.025057,27.510989,-0.065934
2,0K0Xq95kh0c51N0x2pB1,ratio_cycles_operating_hours,0.962627,1.874231,0.183654
3,0K0Xq95kh0c51N0x2pB1,average_cycle_length_hours,0.880879,0.506319,-0.024643
4,0SPSfQIq_F2spSYR_Z4E,operating_hours,0.991551,12.553516,-0.710137
...,...,...,...,...,...
1389,zfvIA1q18Lwannw1EVd8,average_cycle_length_hours,0.485676,0.874560,-0.043132
1390,zvNJQfjyMrMydMVP2gld,operating_hours,0.920565,11.442363,-0.504560
1391,zvNJQfjyMrMydMVP2gld,cycles,0.823129,14.560440,-0.362637
1392,zvNJQfjyMrMydMVP2gld,ratio_cycles_operating_hours,0.606764,1.316209,0.047747


### Evaluation of Cycling Behavior With Local Outlier Factor (LOF)

This step applies outlier detection in form of the Local Outlier Factor (LOF) to the slopes and intercepts derived in the previous step. This way, it can be detected if an HP behaves atypically by any of the daily cycling metrics. 

In [4]:
# choose settings for LOF 
lof_neighbors = 15
lof_contamination = 'auto'
min_r2 = 0.4

# define the metrics that should be considered for the outlier detection 
# NOTE: use cycling metrics plus utilization and energy intensity for this for the sake of completeness
metrics = ['operating_hours' , 'cycles', 'ratio_cycles_operating_hours', 'average_cycle_length_hours']
df_lof = helper.calculate_local_outlier_factors_from_regression_parameters(df_fits, metrics, lof_neighbors=lof_neighbors, lof_contamination=lof_contamination, min_r2=min_r2, suppress_prints=False)

# calculate average statistics of R^2 values 
dfr2 = helper.calculate_r2_statistics_from_regression_parameters(df_lof)

# calculate flattened version of evaluation - i.e. one row corresponds to one observation instead of one row per metric and HP
metrics = ['operating_hours' , 'cycles', 'ratio_cycles_operating_hours', 'average_cycle_length_hours'] 
df_grouped = helper.derive_evaluation_from_local_outlier_factors(df_lof, metrics, id_col='ID') # NOTE: the parameter id_col defines which columns should be used to group the data frame

# display results
num_atypical = len(df_grouped[df_grouped['atypical_behavior']==True])
print('Number of atypical HPs detected: {} [{}%]'.format(num_atypical, np.round(num_atypical/len(df_grouped)*100.0, 2)))

display(df_fits)
display(dfr2)
display(df_grouped)


operating_hours - Total Observations: 331
operating_hours - LOF Outliers Detected: 4 [1.21%]
--------------------
cycles - Total Observations: 249
cycles - LOF Outliers Detected: 13 [5.22%]
--------------------
ratio_cycles_operating_hours - Total Observations: 287
ratio_cycles_operating_hours - LOF Outliers Detected: 16 [5.57%]
--------------------
average_cycle_length_hours - Total Observations: 286
average_cycle_length_hours - LOF Outliers Detected: 13 [4.55%]
--------------------
Number of atypical HPs detected: 35 [10.32%]


,ID,metric,R2,intercept,slope
0,0K0Xq95kh0c51N0x2pB1,operating_hours,0.984440,13.485275,-0.624341
1,0K0Xq95kh0c51N0x2pB1,cycles,0.025057,27.510989,-0.065934
2,0K0Xq95kh0c51N0x2pB1,ratio_cycles_operating_hours,0.962627,1.874231,0.183654
3,0K0Xq95kh0c51N0x2pB1,average_cycle_length_hours,0.880879,0.506319,-0.024643
4,0SPSfQIq_F2spSYR_Z4E,operating_hours,0.991551,12.553516,-0.710137
...,...,...,...,...,...
1389,zfvIA1q18Lwannw1EVd8,average_cycle_length_hours,0.485676,0.874560,-0.043132
1390,zvNJQfjyMrMydMVP2gld,operating_hours,0.920565,11.442363,-0.504560
1391,zvNJQfjyMrMydMVP2gld,cycles,0.823129,14.560440,-0.362637
1392,zvNJQfjyMrMydMVP2gld,ratio_cycles_operating_hours,0.606764,1.316209,0.047747


,metric,R2_observations_count,R2_median,R2_mean,R2_std
0,operating_hours,350,0.95,0.87,0.21
1,cycles,350,0.71,0.61,0.31
2,ratio_cycles_operating_hours,347,0.82,0.70,0.29
3,average_cycle_length_hours,347,0.80,0.69,0.28


,ID,atypical_behavior,metrics_evaluated,metrics_outlier,metrics_outlier_reasoning
0,0K0Xq95kh0c51N0x2pB1,False,"[operating_hours, ratio_cycles_operating_hours...","[False, False, False]","[nan, nan, nan]"
1,0SPSfQIq_F2spSYR_Z4E,False,"[operating_hours, cycles, ratio_cycles_operati...","[False, False, False, False]","[nan, nan, nan, nan]"
2,0_CA_s_UNagHYgXtHcQj_,False,[operating_hours],[False],[nan]
3,0eq9trggezQhL6u5EcXZ_,False,"[operating_hours, cycles, ratio_cycles_operati...","[False, False, False, False]","[nan, nan, nan, nan]"
4,0krDFEW7If2JtsKzYHGt,True,"[operating_hours, cycles]","[True, False]","[slope: low | intercept: high, nan]"
...,...,...,...,...,...
334,y_o7ZzNPqv6Z84zSIhF,False,"[operating_hours, cycles, ratio_cycles_operati...","[False, False, False, False]","[nan, nan, nan, nan]"
335,yyeGshVUdyomwXT_0_WUW,False,"[operating_hours, ratio_cycles_operating_hours...","[False, False, False]","[nan, nan, nan]"
336,z8Bt2CV_fjA3Sk4dEEqv,False,"[operating_hours, ratio_cycles_operating_hours...","[False, False, False]","[nan, nan, nan]"
337,zfvIA1q18Lwannw1EVd8,False,"[operating_hours, ratio_cycles_operating_hours...","[False, False, False]","[nan, nan, nan]"


### Evaluation of Utilization and Energy Intensity

In [5]:
# select temperature range 
temp_min = 0 # the minimum temperature to be considered for the temperature curves 
temp_max = 12 # the maximum temperature to be considered for the temperature curves
temp_col = 'daily_avgTmp' # column name of the temperature column in the daily metrics data frame
tbase = 20 # base temperature for the calculation of heating degree days

# fuse the cycling metrics data with the meta data on heat pump power and heated floor area
df_energy = df_metrics[['ID', 'date', 'Energy_kWh', temp_col, 'degree_day']].copy().merge(df_meta[['ID', 'HP_Power_kW', 'Heated_Floor_Area']], on='ID', how='left')

# select only observation in the desired temperature range 
df_energy = df_energy[df_energy[temp_col].between(temp_min, temp_max, inclusive='both')]

# calculate utilization and energy intensity (also normalized by heating degree days)
df_energy['utilization'] = helper.calculate_daily_HP_utilization(df_energy['Energy_kWh'], hp_nominal_power=df_energy['HP_Power_kW'], round_decimals=False) / df_energy['degree_day']
df_energy['energy_intensity'] = helper.calculate_daily_energy_intensity(df_energy['Energy_kWh'], floor_area_qm=df_energy['Heated_Floor_Area'], round_decimals=False) / df_energy['degree_day']
df_energy[['utilization', 'energy_intensity']] = df_energy[['utilization', 'energy_intensity']].round(2)

# calculate mean, median and standard deviation of the utilization and energy intensity
df_energy_eval = df_energy.groupby('ID')[['utilization', 'energy_intensity']].agg(['mean', 'median', 'std']).reset_index()
df_energy_eval.columns = ['ID', 'utilization_mean', 'utilization_median', 'utilization_std', 'energy_intensity_mean', 'energy_intensity_median', 'energy_intensity_std']

# perform total evaluation in form of sorting households and assigning them to their percentiles
df_energy_eval['utilization_percentile'] = df_energy_eval['utilization_median'].rank(pct=True)
df_energy_eval['energy_intensity_percentile'] = df_energy_eval['energy_intensity_median'].rank(pct=True)

# NOTE: households with missing data on heat pump power or heated floor area will contain NAN values in the evaluation data frame

# display results 
display(df_energy_eval)

,ID,utilization_mean,utilization_median,utilization_std,energy_intensity_mean,energy_intensity_median,energy_intensity_std,utilization_percentile,energy_intensity_percentile
0,0K0Xq95kh0c51N0x2pB1,3.645978,3.670,0.682839,0.013587,0.01,0.005027,0.817337,0.526455
1,0SPSfQIq_F2spSYR_Z4E,2.860160,2.900,0.521952,0.009904,0.01,0.001264,0.687307,0.526455
2,0_CA_s_UNagHYgXtHcQj_,NaN,NaN,NaN,0.012500,0.01,0.005000,NaN,0.526455
3,0eq9trggezQhL6u5EcXZ_,NaN,NaN,NaN,0.021600,0.02,0.009434,NaN,0.866402
4,0krDFEW7If2JtsKzYHGt,3.269340,4.285,2.729096,0.021181,0.03,0.019145,0.873065,0.947090
...,...,...,...,...,...,...,...,...,...
413,y_o7ZzNPqv6Z84zSIhF,1.238459,1.230,0.571912,0.007541,0.01,0.004679,0.241486,0.526455
414,yyeGshVUdyomwXT_0_WUW,5.301545,5.290,1.132896,0.028584,0.03,0.006833,0.919505,0.947090
415,z8Bt2CV_fjA3Sk4dEEqv,4.851194,4.805,1.021211,0.009903,0.01,0.001502,0.894737,0.526455
416,zfvIA1q18Lwannw1EVd8,3.605548,3.500,1.003397,0.014086,0.01,0.005500,0.795666,0.526455
